# Proyecto SQL

En esta etapa del proyecto final, utilizaremos SQL para realizar un análisis sobre el comportamiento de venta de libros de una starup. 

Dentro del estudio, buscaremos identificar aspectos claves para el conocimiento y entendimiento de diferentes factores referentes al dinamismo de los libros. 

## Importamos las librerías 

In [1]:


# Importar librerías
import pandas as pd
from sqlalchemy import create_engine, inspect

# Configuración de conexión
db_config = {
    'user': 'practicum_student',  # usuario
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',  # contraseña
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,  # puerto
    'db': 'data-analyst-final-project-db'  # nombre de la base de datos
}

# Crear string de conexión
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

# Conexión al motor
engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

# 1. Listar todas las tablas disponibles
inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tablas disponibles en la base de datos:")
print(tables)


# 2. Loop para leer y mostrar las primeras filas de cada tabla
for table in tables:
    print(f"\nMostrando primeras filas de la tabla: {table}")
    query = f"SELECT * FROM {table} LIMIT 5;"
    df = pd.read_sql(query, con=engine)
    print(df.head())


Tablas disponibles en la base de datos:
['ratings', 'advertisment_costs', 'authors', 'orders', 'reviews', 'visits', 'books', 'users', 'publishers']

Mostrando primeras filas de la tabla: ratings
   rating_id  book_id       username  rating
0          1        1     ryanfranco       4
1          2        1  grantpatricia       2
2          3        1   brandtandrea       5
3          4        2       lorichen       3
4          5        2    mariokeller       2

Mostrando primeras filas de la tabla: advertisment_costs
   id  sourceid         dt   costs
0   1         1 2017-06-01  $75.20
1   2         1 2017-06-02  $62.25
2   3         1 2017-06-03  $36.53
3   4         1 2017-06-04  $55.00
4   5         1 2017-06-05  $57.08

Mostrando primeras filas de la tabla: authors
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Bren

Con lo anterior, revisamos las primeras filas de cada tabla para conocer la información contenida en cada una de ellas. 

## Libros publicados después del 1 de enero de 2000

Usando SQL, hallaremos el total de numeros públicos después del 01/01/2020

In [2]:

query = """
SELECT COUNT(*) AS num_books
FROM books
WHERE publication_date > '2000-01-01';
"""

df = pd.read_sql(query, con=engine)
print(df)


   num_books
0        819


En total, se publicaron 819 libros 

## Número de reseñas de usuarios y la calificación promedio de cada libro

In [3]:
query = """
SELECT 
    r.book_id,
    COUNT(DISTINCT r.review_id) AS num_reviews,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM reviews r
JOIN ratings rt ON r.book_id = rt.book_id
GROUP BY r.book_id
ORDER BY r.book_id;
"""

df = pd.read_sql(query, con=engine)
print(df.head(20))

    book_id  num_reviews  avg_rating
0         1            2        3.67
1         2            1        2.50
2         3            3        4.67
3         4            2        4.50
4         5            4        4.00
5         6            4        3.50
6         7            3        3.00
7         8            3        4.00
8         9            2        4.25
9        10            3        3.20
10       11            3        4.25
11       12            4        4.00
12       13            2        3.00
13       14            1        4.50
14       15            5        4.39
15       16            3        4.50
16       17            4        5.00
17       18            2        4.00
18       19            4        4.00
19       20            2        5.00


## Editorial con mayor publicaciones de libros con más de 50 páginas 

In [4]:
query = """
SELECT p.publisher_id,
       p.publisher AS publisher_name,
       COUNT(b.book_id) AS num_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY num_books DESC
LIMIT 1;
"""

df = pd.read_sql(query, con=engine)
print(df)

   publisher_id publisher_name  num_books
0           212  Penguin Books         42


Descartando folletos (O pulicaciones similares) tenemos que Penguin Books ha publicado 42 con más de 50 páginas. 

## Autor con más alta calificación promedio del libro

In [5]:
query = """
WITH book_stats AS (
    SELECT 
        book_id,
        COUNT(rating) AS num_ratings,
        AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating) >= 50
)
SELECT 
    a.author_id,
    a.author AS author_name,
    ROUND(AVG(bs.avg_rating), 2) AS author_avg_rating
FROM book_stats bs
JOIN books b ON bs.book_id = b.book_id
JOIN authors a ON b.author_id = a.author_id
GROUP BY a.author_id, a.author
ORDER BY author_avg_rating DESC
LIMIT 1;
"""

df = pd.read_sql(query, con=engine)
print(df)

   author_id                 author_name  author_avg_rating
0        236  J.K. Rowling/Mary GrandPré               4.28


¡También votaría por J.K. Rowling! con al menos 50 calificaciones, tiene un promedio de calificación de 4.28. 

## Promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

In [6]:
query = """
WITH active_users AS (
    SELECT 
        username,
        COUNT(rating_id) AS num_ratings
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
),
user_reviews AS (
    SELECT 
        username,
        COUNT(review_id) AS num_reviews
    FROM reviews
    GROUP BY username
)
SELECT 
    ROUND(AVG(ur.num_reviews), 2) AS avg_reviews_per_user
FROM active_users au
LEFT JOIN user_reviews ur ON au.username = ur.username;
"""

df = pd.read_sql(query, con=engine)
print(df)

   avg_reviews_per_user
0                 24.33
